# Feature generation

The first thing I want to note here is that model selection was a very important criteria for feature generation, but I am going to generally break up the documentation of model selection and feature generation for organizational purposes. But if you want skip ahead to to the next notebook feel free!

## Widending that data

Before getting into real feature generation, I made some alterations to the data. As mentioned in the last notebook, the dataset was really big. I wanted to keep the data in it's native format for EDA. It is much easier to column when they hold single objects than it is to analyze semi-structured elements. However, for the rest of the pipeline I need to otpimize the data, so I reformatted the data utizlining PostgreSQL array columns in the scripts _06_traces_to_db_partioning and _07_traces_to_segements.py . I went ahead and threw out the fields I determined I wouldn't use, but other than that no mathemetical transfomrated occured at this step. All traces of less than 15 minutes were not included in the output data set. 

Here is brief summary of what each script did and how:

- _06_traces_db_partitioning.py
 - Partitions aircraft tracking data table into smaller tables based on first two characters of ICAO aircraft identifier (hex prefixes '00' to 'ff')
 - Creates up to 256 separate tables with identical schema
 - Transfers data in batches of 100,000 rows with indexes on timestamp and ICAO columns
 - Can process either full hex range (00-ff) or specific subsets

- _07_traces_to_segments.py
 - Processes aircraft tracking data across different hex-partitioned tables
 - Implements multi-level parallelization: thread pooling for hex partitions and process pooling for aircraft (ICAO codes)
 - Three main stages: extraction of raw trace data, transformation of flight segments based on timestamp gaps, and loading of processed data

The key tools used to perfrom the transformation have already been mentioned in the preceeding notebook and include same databse and parellelization tools.

Let's go ahead and do our imports for our notebok.

#### Dependancies and config

In [1]:
import pandas as pd

### View of segmented data

In [2]:
# output what the segmented data loks like

# read in sample data
df = pd.read_parquet('../data/_07_traces_to_segments/segments_sample.parquet')

display(df.head())

,icao,start_timestamp,duration,time_offsets,nav_heading,longitude,vertical_rate_baro,latitude,ground_speed,flags,altitude_baro,nav_altitude_mcp,track,indicated_airspeed,roll_angle,squawk,flight,category,emergency
0,407fb9,2024-08-01 15:53:47.540000+00:00,0 days 00:34:07.310000,"[0, 8430000, 48120000, 66400000, 119710000, 13...",None,"[2.237915, 2.230303, 2.197187, 2.180001, 2.131...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[53.147003, 53.143295, 53.127274, 53.118942, 5...","[148.7, 148.7, 147.9, 147.9, 147.9, 147.9, 148...","[3, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1600, 1600, 1600, 1600, 1600, 1600, 1600, 160...","[35008, 38000, 33008, 33008, 20000, 20000]","[231.30, 231.30, 231.00, 231.00, 231.00, 231.0...",None,"[0.00, 0.00, 0.00, -0.20, 0.00, -0.20, 0.00, -...","[7420, 7420, 7420, 7420, 7420, 7420, 7420, 742...",None,None,None
1,407fb9,2024-08-01 17:27:53.700000+00:00,0 days 00:19:51.340000,"[0, 1080000, 10140000, 11990000, 13470000, 149...",None,"[1.287912, 1.287994, 1.288304, 1.288376, 1.288...","[-960, 448, -192, 0, 0, 0, -320, -256, 832, 13...","[52.675347, 52.675415, 52.675859, 52.675919, 5...","[9.9, 12.5, 12.7, 8.1, 8.1, 3.7, 1.2, 0.7, 2.1...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[100, 100, 75, 75, 75, 75, 50, 75, 125, 200, 2...","[35008, 35008, 9008, 35008]","[45.00, 28.60, 18.40, 29.70, 29.70, 56.30, 90....","[264, 274, 258, 260, 260, 259, 259]","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, -4.20, -8...","[7420, 7420, 7420, 7420, 7420, 7420, 7420, 742...",None,None,None
2,407fb9,2024-08-01 17:58:07.680000+00:00,0 days 00:28:40.530000,"[0, 3300000, 41850000, 44290000, 68970000, 768...",None,"[2.283325, 2.283249, 2.288971, 2.288513, 2.271...","[960, 960, 960, 576, 704, 640, 704, 704, 768, ...","[53.051651, 53.051651, 53.059891, 53.060760, 5...","[89.0, 89.0, 89.0, 130.9, 130.7, 131.4, 132.4,...","[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[50, 50, 475, 500, 800, 875, 900, 1000, 1075, ...",[19008],"[0.60, 0.60, 0.60, 247.60, 240.70, 239.30, 238...","[250, 250, 247, 247, 247, 247, 247, 247]","[-19.50, -19.50, -19.50, -10.60, -10.60, -10.6...","[7420, 7420, 7420, 7420, 7420, 7420, 7420, 742...",None,A7,None
3,0200ae,2024-08-01 10:47:06.580000+00:00,0 days 02:52:23.940000,"[0, 75440000, 145750000, 180620000, 240330000,...","[224.30, 224.30, 224.30, 224.30, 224.30, 224.3...","[6.110476, 6.110476, 6.110475, 6.110475, 6.110...","[64, 0, 0, 0, 64, 64, 64, 64, 128, 256, 448, 6...","[46.233751, 46.233751, 46.233753, 46.233753, 4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9008, 9008, 9008, 9008, 9008, 9008, 9008, 900...","[135.00, 135.00, 135.00, 135.00, 135.00, 135.0...","[118, 118, 118, 118, 146, 146, 163, 163, 163, ...","[-0.50, -0.50, -0.50, -0.50, -0.50, -0.50, -0....","[5751, 5751, 5751, 5751, 5751, 5751, 5751, 575...",None,None,none
4,407fc1,2024-08-01 05:16:41.990000+00:00,0 days 01:57:33.120000,"[0, 2610000, 6920000, 25100000, 31650000, 3554...","[75.23, 75.23, 59.77, 59.77, 59.77, 59.77, 59....","[-0.379715, -0.379677, -0.379601, -0.379344, -...","[96, 0, -1376, -1376, -1376, -1376, -1376, -13...","[51.880293, 51.880281, 51.880246, 51.880199, 5...","[2.2, 2.8, 2.2, 1.4, 1.9, 1.1, 0.0, 0.0, 0.0, ...","[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4992, 4992, 4992, 4992, 4992, 4992, 4992, 499...","[281.20, 275.60, 264.40, 219.40, 219.40, 216.6...","[256, 256, 262, 262, 262, 262, 262, 262, 152, ...","[0.20, 0.20, 0.50, 0.50, 0.70, 0.70, 0.70, 0.7...","[7305, 7305, 7305, 7305, 7305, 7305, 7305, 730...",None,None,None


# Feature generation strategy

This feature generation approach fundamentally differs from traditional flight data analysis methods by preserving the inherent time-series nature of aircraft behavior while enabling both global pattern recognition and local anomaly detection. Unlike conventional methods that often reduce flight segments to aggregate statistics (like mean, standard deviation, or min/max values), this approach calculates instantaneous rates of change through forward differences, maintaining the temporal relationship between consecutive measurements and capturing the dynamic evolution of flight parameters.

The overlapping 50-point segment design serves multiple crucial purposes. First, it ensures continuous coverage of flight patterns, preventing blind spots that could occur at fixed-window boundaries where important maneuvers or anomalies might be split. Second, by deriving features like ground acceleration, vertical acceleration, turn rates, and climb/descent accelerations using forward differences, the method captures not just the magnitude of changes but their progression over time. This is particularly important for ASDB flight data, where both commercial and general aviation patterns need to be analyzed, as it preserves the sequential nature of aircraft maneuvers while remaining computationally feasible. The approach strikes a balance between granular anomaly detection (such as unusual accelerations within otherwise normal flight patterns) and the practical constraints of processing large-scale flight data, making it well-suited for use with deep learning techniques like autoencoders that can simultaneously learn normal patterns while identifying subtle deviations.

- Preserves time-series relationships
  - Uses forward differences instead of statistical aggregates
  - Enables detection of patterns and anomalies simultaneously
  - Maintains temporal sequence information

- Uses overlapping 50-point segments
  - Provides continuous coverage of flight patterns
  - Works within memory constraints
  - Accommodates variable sampling rates in ASDB data

- Handles real-world data challenges
  - Explicit rules for null values
  - Clear gap handling procedures
  - Defined sequence ending protocols
  - Works across different aircraft types

- Focuses on physics-based derived features
  - Ground and vertical accelerations
  - Turn rates
  - Climb/descent accelerations
  - Direct capture of aircraft dynamics

- Designed for versatility
  - Works with commercial aviation
  - Handles general aviation patterns
  - Captures maneuver-specific details
  - Processes diverse flight behaviors

- Optimized for autoencoder + HDBSCAN
  - Enables simultaneous pattern learning
  - Supports anomaly detection
  - Provides interpretable latent space
  - Works with high-dimensional data

- Avoids traditional method limitations
  - Better than HMMs with continuous data
  - Superior to isolation forests
  - Maintains temporal relationships
  - Handles high-dimensional spaces effectively

# Feature Transformations

## Input Data Arrays
- `flight_data['time_offsets']`: Microsecond time offsets
- `flight_data['vertical_rate_baro']`: Vertical rate in feet/minute
- `flight_data['ground_speed']`: Ground speed in knots
- `flight_data['track']`: Track/heading in degrees (0-360)
- `flight_data['altitude_baro']`: Barometric altitude in feet

## Segmentation Logic
Data is grouped by ICAO. Each ICAO\'s data is processed for valid 50-point segments:

- Main segments: Points 1-50, 51-100, etc.
- Overlapping segments: Points 25-75, 75-125, etc.
- End handling: If sequence ends with <50 points, take last 50 points
- Gap handling: If 10+ second gap found, take previous 50 points if available

## Validity Check
Data Quality requirements:

- Null values break sequences
- All arrays must maintain 50 points
- No duplicate timestamps (take first point if duplicates exist)

## Derived Feature Calculations
- Using forward differences for all calculations:

### Ground Acceleration (ground_accels)
- For point i:
  - Δt = (time_offsets[i+1] - time_offsets[i]) / 1_000_000  # convert to seconds
  - ground_accels[i] = (ground_speed[i+1] - ground_speed[i]) / Δt
- Units: knots/second

### Vertical Acceleration (vertical_accels)
- For point i:
  - Δt = (time_offsets[i+1] - time_offsets[i]) / 1_000_000
  - vertical_accels[i] = (vertical_rate_baro[i+1] - vertical_rate_baro[i]) / Δt
- Units: feet/min²

### Turn Rate (turn_rates)
- For point i:
  - Δt = (time_offsets[i+1] - time_offsets[i]) / 1_000_000
  - heading_diff = track[i+1] - track[i]

### Handle 0/360 wraparound
- if heading_diff > 180:
  - heading_diff -= 360
- elif heading_diff < -180:
  - heading_diff += 360
       
  - turn_rates[i] = heading_diff / Δt
- Units: degrees/second

### Climb/Descent Acceleration (climb_descent_accels)
- For point i:
  - Δt1 = (time_offsets[i+1] - time_offsets[i]) / 1_000_000
  - Δt2 = (time_offsets[i+2] - time_offsets[i+1]) / 1_000_000

  - first_deriv = (altitude_baro[i+1] - altitude_baro[i]) / Δt1
  - second_deriv = (altitude_baro[i+2] - altitude_baro[i+1]) / Δt2

  - climb_descent_accels[i] = (second_deriv - first_deriv) / Δt1
- Units: feet/second²

### Notes on Feature Generation
- Last point(s) copy previous value to maintain 50-point arrays
- No smoothing applied to preserve raw change measurements
- Originally, a data verification step was in place to check for physically unlikely data points
  - this check proved a little too aggressive
  - likely filtering a lot of values due to sensor error
  - ended up with what looked like cruising data for all clusters
  - and it was making all my visuslization in the final notebook super boring, so I commented it out for now

In [5]:
# extract and print of what the data looks after feature engineering
df = pd.read_parquet('../data/_11_analysis_conclusions/autoencoder_training_sample.parquet')

display(df.head())

,segment_id,icao,start_timestamp,segment_duration,point_count,vertical_rates,ground_speeds,headings,altitudes,time_offsets,vertical_accels,ground_accels,turn_rates,climb_descent_accels
0,11818037,ad6182,2024-08-01 17:15:23.060000+00:00,0 days 00:14:20.120000,50,"[1856.0, 1792.0, 1664.0, 1600.0, 1600.0, 1600....","[374.3, 375.2, 376.2, 377.3, 379.3, 380.3, 382...","[265.2, 265.4, 265.4, 265.3, 265.3, 265.2, 265...","[14850.0, 15175.0, 15550.0, 15875.0, 16200.0, ...","[0, 12850000, 25180000, 37600000, 50670000, 64...","[-4.980544747081712, -10.38118410381184, -5.15...","[0.07003891050583481, 0.08110300081103, 0.0885...","[0.015564202334629466, 0.0, -0.008051529790657...","[0.3985833860203841, -0.3443757894963926, -0.1..."
1,10468011,a2baea,2024-08-01 15:12:15.510000+00:00,0 days 00:12:58.260000,50,"[-64.0, -64.0, 128.0, 0.0, 0.0, 0.0, 64.0, -38...","[175.7, 175.7, 174.7, 175.7, 171.6, 174.7, 173...","[281.8, 281.8, 281.9, 281.8, 281.8, 281.9, 281...","[7975.0, 7975.0, 8000.0, 7975.0, 7975.0, 7975....","[0, 19640000, 39350000, 58980000, 78770000, 98...","[0.0, 9.74124809741248, -6.5206316861946005, 0...","[0.0, -0.050735667174023336, 0.050942435048395...","[0.0, 0.005073566717400603, -0.005094243504837...","[0.06458206106673031, -0.1289676588310739, 0.0..."
2,8911791,485875,2024-08-01 18:48:54.270000+00:00,0 days 00:01:22.410000,50,"[3456.0, 3456.0, 3456.0, 3392.0, 3328.0, 3264....","[145.1, 143.0, 141.8, 141.6, 141.6, 140.6, 140...","[11.9, 11.7, 11.4, 11.0, 11.0, 11.1, 10.7, 10....","[600.0, 700.0, 775.0, 875.0, 950.0, 1050.0, 11...","[0, 1580000, 3280000, 4680000, 6130000, 745000...","[0.0, 0.0, -45.714285714285715, -44.1379310344...","[-1.3291139240506293, -0.7058823529411699, -0....","[-0.12658227848101333, -0.1764705882352935, -0...","[-12.135121633976453, 16.065249629263473, -14...."
3,9962051,888159,2024-08-01 08:08:36+00:00,0 days 00:12:22.300000,50,"[1024.0, 576.0, 128.0, 512.0, 960.0, 1024.0, 5...","[442.8, 443.8, 445.9, 449.9, 449.9, 448.9, 447...","[266.5, 266.5, 266.4, 266.3, 266.3, 266.3, 266...","[25375.0, 25525.0, 25575.0, 25625.0, 25675.0, ...","[0, 11500000, 21740000, 37710000, 41690000, 56...","[-38.95652173913044, -43.75, 24.04508453350031...","[0.08695652173913043, 0.20507812499999667, 0.2...","[0.0, -0.00976562500000222, -0.006261740763930...","[-0.7096231096408318, -0.17108809746423959, 0...."
4,10414802,a24c01,2024-08-01 15:55:39.390000+00:00,0 days 00:06:19.390000,50,"[-1984.0, -2048.0, -1856.0, -2112.0, -2048.0, ...","[376.9, 374.0, 371.2, 367.2, 365.8, 363.5, 361...","[219.9, 220.1, 220.1, 219.6, 221.0, 224.3, 229...","[21775.0, 21400.0, 21025.0, 20650.0, 20275.0, ...","[0, 10960000, 22420000, 33970000, 44500000, 48...","[-5.839416058394161, 16.75392670157068, -22.16...","[-0.26459854014598333, -0.24432809773124006, -...","[0.018248175182480713, 0.0, -0.043290043290043...","[0.13620578267774194, 0.022249617929560917, -0..."
